## Etherscan Notebook 

Looking at how to derive price for tokens and gas fees by pool.  Pool example can be found [here](https://app.uniswap.org/explore/pools/ethereum/0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640)

In [6]:
import requests
import time 
import os
import pandas as pd
import numpy as np
from web3 import Web3

import src.arbutils as arbutils


In [4]:
# change the active path to the parent directory 
if True: 
    print("Moving active path to parent directory")
    os.chdir('..')
    print(os.getcwd())

Moving active path to parent directory
/Users/das/DATASCI210/arbitrage_playground


In [3]:
API_KEY =  os.getenv('GRAPH_API_KEY')
if not API_KEY:
    print("No GRAPH_API_KEY found")
else:
    print("Found API Key!")

Found API Key!


In [7]:
POOL0_ADDRESS="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" # USDC / WETH (0.05%) 
POOL0_TXN_FEE = 0.0005
POOL1_ADDRESS="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" # USDC / WETH (0.3%)
POOl1_TXN_FEE = 0.003

GWEI_SCALER = 1e9

In [8]:
p0 = arbutils.thegraph_request(API_KEY, 
                            POOL0_ADDRESS,
                            new_date=None, 
                            old_date=None, 
                            data_path=None, 
                            checkpoint_file=None)

p1 = arbutils.thegraph_request(API_KEY, 
                            POOL1_ADDRESS,
                            new_date=None, 
                            old_date=None, 
                            data_path=None, 
                            checkpoint_file=None)

merged_pools = arbutils.merge_pool_data_v2(p0, 
                                           POOL0_TXN_FEE, 
                                           p1, 
                                           POOl1_TXN_FEE)


Starting from timestamp: 1736878437
query number 0 1736878437 1736882037
Batch 1, Total swaps: 188, Avg Fetch Time: 0.5s
query number 1 1736881919 1736882037
Starting from timestamp: 1736878438
query number 0 1736878438 1736882038
Batch 1, Total swaps: 5, Avg Fetch Time: 0.2s
query number 1 1736881175 1736882038
Are there any NaNs in the DataFrame? False


/Users/das/DATASCI210/arbitrage_playground/src/arbutils.py:540: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  both_pools = both_pools.ffill().reset_index(drop=True)


In [9]:
p0.columns

Index(['transactionHash', 'datetime', 'timeStamp', 'sqrtPriceX96',
       'blockNumber', 'gasPrice', 'gasUsed', 'tick', 'amount0', 'amount1',
       'liquidity'],
      dtype='object')

In [10]:
p0.head()

,transactionHash,datetime,timeStamp,sqrtPriceX96,blockNumber,gasPrice,gasUsed,tick,amount0,amount1,liquidity
0,0x38816e2410db7683d3d0b7533096e85f4a341af972c5...,2025-01-14 18:13:59+00:00,1736878439,1.398392e+33,21624449,1.066782e+10,0.0,195579.0,178.406690,-0.055551,-1.0
1,0xe852dbce0570a87b242aa1b43464397e9a87c4cf5d50...,2025-01-14 18:13:59+00:00,1736878439,1.398380e+33,21624449,1.243841e+10,0.0,195579.0,2024.384283,-0.630334,-1.0
2,0x000eff2dc98216db4fc36fc83393be7cc47c1e64525f...,2025-01-14 18:14:23+00:00,1736878463,1.398349e+33,21624451,1.198704e+10,0.0,195579.0,5197.473245,-1.618290,-1.0
3,0xa2b90db051d488eebbdf430171d47b54ebee2e1f9a73...,2025-01-14 18:14:23+00:00,1736878463,1.397916e+33,21624451,1.220451e+10,0.0,195573.0,72080.699152,-22.435681,-1.0
4,0x3062ec26c7d593ab7978c3a196fe14d244f3426ae5c4...,2025-01-14 18:14:47+00:00,1736878487,1.397784e+33,21624453,1.204943e+10,0.0,195571.0,21000.000000,-6.533749,-1.0


Now that I have information at the transaction level.  I would like to parse specific transactions to extract the sqrtPriceX96 value which is the only value missing for our current model.  Also, assuming that I am doing this for inference, I only need transactions that fit the following criteria: (1) at least one transactions from each pool (because of the forward fill), (2) at least 10 transactions total (to ensure we can do lags and moving averages for feature extraction). 

For now, I will just truncate both to use only the last ten from each pool.  This is a shortcut for inference only that allows redunancy in the fetch for data from logs, allows for forward fill of known values, etc.

In [11]:
p0 = p0.iloc[-10:]
p1 = p1.iloc[-10:]

**Get price within the Pool in ETH/USDC**

To derive the price for the pool in ETH/USDC, you must use the sqrtPriceX96 value, which is the pool price immediately after the transaction takes place (including slippage).  You can see below that there is almost always a descrepency, but its not always enough to over come transaction and gas fees (see below).

In [13]:
# row to pick for the swap...used just for the example.
pool0_price_in_USDC_per_ETH  = ((p0["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1
pool1_price_in_USDC_per_ETH  = ((p1["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1

print(f"Pool 0 Price in USDC per ETH (at Tx: 0x...{p0['transactionHash'].iloc[0][-4:]}): ${pool0_price_in_USDC_per_ETH:.2f}")
print(f"Pool 1 Price in USDC per ETH (at Tx: 0x...{p1['transactionHash'].iloc[0][-4:]}): ${pool1_price_in_USDC_per_ETH:.2f}")
print(f"Difference in price: ${pool1_price_in_USDC_per_ETH-pool0_price_in_USDC_per_ETH:.2f}")

Pool 0 Price in USDC per ETH (at Tx: 0x...0638): $3223.10
Pool 1 Price in USDC per ETH (at Tx: 0x...2131): $3214.52
Difference in price: $-8.58


**Get gas fees in ETH**

Gas fees for a transaction include all the 'work' done. There is a rate of fee per unit of work (i.e. gasPrice) and then there is the work done (i.e. gasUsed).  gasPrice and gasUsed is in gwei which is 1e9 of an ETH.  so to convert to eth, each value needs to be converted with the 1e9 scaling.

In [14]:
gas_price_eth_tokens_per_unit = int(p0['gasPrice'].iloc[0])/GWEI_SCALER
gas_used_units = int(p0['gasUsed'].iloc[0]) / GWEI_SCALER
gas_fees_eth_tokens  = gas_price_eth_tokens_per_unit* gas_used_units
gas_fees_usdc_tokens = pool0_price_in_USDC_per_ETH * gas_fees_eth_tokens 
print(f"Gas Price in ETH per unit: {gas_price_eth_tokens_per_unit}")
print(f"Gas Used in GWEI units for Uniswap Transaction: {gas_used_units}")
print(f"Gas fees for this Transaction in ETH: {gas_fees_eth_tokens:.5f} (ETH)")
print(f"Gas fees for this Transaction in USDC: ${gas_fees_usdc_tokens:.2f} (USDC)")

Gas Price in ETH per unit: 8.199025182
Gas Used in GWEI units for Uniswap Transaction: 0.0
Gas fees for this Transaction in ETH: 0.00000 (ETH)
Gas fees for this Transaction in USDC: $0.00 (USDC)
